
# DSC212: Modularity on the Karate Club Graph — Spectral Recursive Bisection

**What this notebook does (ready-to-run):**
- Implements *spectral modularity* bipartitioning and extends it to multiple communities via **recursive bisection**.
- Visualizes the Karate Club network after each split using a **fixed spring layout** and unique colors per community.
- Computes node metrics after each split: **degree centrality**, **betweenness centrality**, **closeness centrality**, and **clustering coefficient** (using NetworkX implementations).
- Plots how each node's metrics evolve across iterations.
- Provides a short discussion of results and which nodes remain central as communities emerge.

**Requirements:** `networkx`, `numpy`, `scipy`, `matplotlib`, `pandas`. These are standard scientific Python tools.


In [ ]:

# Imports
import networkx as nx
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import pandas as pd
from collections import deque, defaultdict
import os

# Make sure output dir exists
OUT_DIR = '/mnt/data/karate_modularity_outputs'
os.makedirs(OUT_DIR, exist_ok=True)

# plotting defaults
plt.rcParams['figure.figsize'] = (8,5)


In [ ]:

# Load the Karate Club graph (NetworkX has it built-in)
G = nx.karate_club_graph()  # nodes are 0..33 by default in NetworkX
n = G.number_of_nodes()
m = G.number_of_edges()
print(f"Karate Club loaded: n={n}, m={m}")


In [ ]:

def modularity_matrix_for_subset(G, subset_nodes, k_global=None, m_global=None):
    """Compute restricted modularity matrix B(C) for nodes in subset_nodes.
       Returns B as a NumPy array and the mapping index->node and node->index."""
    nodes = list(subset_nodes)
    idx_of = {v:i for i,v in enumerate(nodes)}
    if k_global is None:
        k = np.array([G.degree(v) for v in nodes], dtype=float)
    else:
        k = np.array([k_global[v] for v in nodes], dtype=float)
    if m_global is None:
        m = G.number_of_edges()
    else:
        m = m_global
    # adjacency block
    A = np.zeros((len(nodes), len(nodes)), dtype=float)
    for i,u in enumerate(nodes):
        for j,v in enumerate(nodes):
            if G.has_edge(u,v):
                A[i,j] = 1.0
    # global degree vector for the whole graph to compute expected term uses global degrees
    # need full degree vector across all nodes to compute expected kikj/(2m)
    # we'll pass in k_global for convenience; if missing, compute from G over full node set.
    if k_global is None:
        k_full = {v: G.degree(v) for v in G.nodes()}
    else:
        k_full = k_global
    # Build kk^T/(2m) for the selected indices using global degrees
    K = np.array([k_full[v] for v in nodes], dtype=float)
    expected = np.outer(K, K) / (2.0*m)
    B = A - expected
    return B, nodes, idx_of

def leading_eigenpair(B):
    """Return the leading eigenvalue and eigenvector of real symmetric B."""
    # using scipy.linalg.eigh for symmetric matrices
    vals, vecs = la.eigh(B)
    # eigh returns ascending eigenvalues; pick the largest
    idx = np.argmax(vals)
    return float(vals[idx]), vecs[:, idx]

def spectral_bipartition(B, nodes):
    """Given B and corresponding nodes list, compute sign split using leading eigenvector.
       Returns two lists: nodes_positive, nodes_negative, and the eigenvalue."""
    lam, u = leading_eigenpair(B)
    # if largest eigenvalue <= 0, no split
    if lam <= 0:
        return None, None, lam
    s = np.where(u > 0, 1, -1)
    nodes_pos = [nodes[i] for i in range(len(nodes)) if s[i] == 1]
    nodes_neg = [nodes[i] for i in range(len(nodes)) if s[i] == -1]
    return nodes_pos, nodes_neg, lam

# quick sanity check: compute B for full graph and test leading eigenpair
k_global = {v: G.degree(v) for v in G.nodes()}
B_full, nodes_full, _ = modularity_matrix_for_subset(G, list(G.nodes()), k_global=k_global, m_global=m)
lam_full, u_full = leading_eigenpair(B_full)
lam_full


In [ ]:

def recursive_modularity_bisection(G, verbose=True):
    """Perform recursive bisection using restricted modularity matrix B(C).
       Returns:
         - communities: list of lists of nodes (final partition)
         - snapshots: list of dicts with keys 'communities' and 'labels' after each split (for visualization/metrics)
    """
    # We'll maintain a queue of candidate communities to split.
    # Also keep a global iteration counter and snapshots.
    queue = deque()
    initial = list(G.nodes())
    queue.append(initial)
    communities = []
    snapshots = []  # each snapshot: {'communities': [list,...], 'labels': {node:comm_id}}
    # initial snapshot: one community
    cur_comm_list = [initial.copy()]
    labels = {v: 0 for v in G.nodes()}
    snapshots.append({'communities': [initial.copy()], 'labels': labels.copy(), 'eigenvalue': None, 'split_nodes': None})
    comm_id_counter = 1  # used to give new community ids for labelling in snapshots (visualization only)
    while queue:
        C = queue.popleft()
        Bc, nodesC, idx_of = modularity_matrix_for_subset(G, C, k_global=k_global, m_global=m)
        lam, u = leading_eigenpair(Bc)
        if verbose:
            print(f"Considering community of size {len(C)}; leading eigenvalue = {lam:.6f}")
        if lam <= 0:
            # cannot split; this becomes a final community
            communities.append(C)
            continue
        # else perform split
        pos, neg, _ = spectral_bipartition(Bc, nodesC)
        if (not pos) or (not neg):
            # degenerate split; treat as indivisible
            communities.append(C)
            continue
        # update current list of communities for snapshotting: replace C with pos and neg
        # find where C sits in cur_comm_list and replace
        replaced = False
        for i,comm in enumerate(cur_comm_list):
            if set(comm) == set(C):
                cur_comm_list[i] = pos
                cur_comm_list.insert(i+1, neg)
                replaced = True
                break
        if not replaced:
            # safe fallback: append both
            cur_comm_list.append(pos)
            cur_comm_list.append(neg)
        # update labels for snapshot (assign unique ids)
        labels = {}
        for cid,comm in enumerate(cur_comm_list):
            for v in comm:
                labels[v] = cid
        snapshots.append({'communities': [c.copy() for c in cur_comm_list], 'labels': labels.copy(), 'eigenvalue': lam, 'split_nodes': C})
        # enqueue subcommunities for further splitting
        queue.append(pos)
        queue.append(neg)
    return communities, snapshots

# Run recursive bisection and collect snapshots
communities, snapshots = recursive_modularity_bisection(G, verbose=True)
print('\nFinal communities ({}):'.format(len(communities)))
for i,c in enumerate(communities):
    print(f'Community {i}: size={len(c)} nodes={sorted(c)}')


In [ ]:

# Fixed layout for consistent plotting across snapshots
pos = nx.spring_layout(G, seed=42)

def plot_snapshot(G, snapshot, iter_idx, out_dir=OUT_DIR):
    labels = snapshot['labels']
    comms = snapshot['communities']
    # assign a color to each community index
    num_comms = len(comms)
    cmap = plt.get_cmap('tab20')
    node_colors = [cmap(labels[n] % 20) for n in G.nodes()]
    plt.figure(figsize=(7,6))
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=350)
    nx.draw_networkx_edges(G, pos, alpha=0.6)
    nx.draw_networkx_labels(G, pos, {n:str(n) for n in G.nodes()}, font_size=9)
    plt.title(f'Iteration {iter_idx}: {num_comms} communities, lambda={snapshot.get("eigenvalue"):.4f}')
    plt.axis('off')
    fname = os.path.join(out_dir, f'snapshot_iter_{iter_idx:02d}.png')
    plt.savefig(fname, bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()

# Plot all snapshots
for i,snap in enumerate(snapshots):
    plot_snapshot(G, snap, i)


In [ ]:

# Compute metrics after each snapshot and store them in DataFrames
metric_names = ['degree','betweenness','closeness','clustering']
metric_history = {name: [] for name in metric_names}
iterations = []

for i,snap in enumerate(snapshots):
    iterations.append(i)
    # For these metrics, we compute on the full graph (as required) but after the split we
    # annotate which community each node belongs to (snap['labels']).
    deg = nx.degree_centrality(G)  # dict node->value
    btw = nx.betweenness_centrality(G, normalized=True)
    clo = nx.closeness_centrality(G)
    clus = nx.clustering(G)
    # convert to pandas Series ordered by node id
    nodes_sorted = sorted(G.nodes())
    metric_history['degree'].append(pd.Series([deg[n] for n in nodes_sorted], index=nodes_sorted))
    metric_history['betweenness'].append(pd.Series([btw[n] for n in nodes_sorted], index=nodes_sorted))
    metric_history['closeness'].append(pd.Series([clo[n] for n in nodes_sorted], index=nodes_sorted))
    metric_history['clustering'].append(pd.Series([clus[n] for n in nodes_sorted], index=nodes_sorted))

# Build DataFrames: index = iterations, columns = node ids
metrics_dfs = {}
for name in metric_names:
    df = pd.DataFrame(metric_history[name], index=iterations)
    df.index.name = 'iteration'
    metrics_dfs[name] = df
    # save CSV for convenience
    df.to_csv(os.path.join(OUT_DIR, f'metric_{name}.csv'))

# Show table shapes
for name,df in metrics_dfs.items():
    print(f"Metric {name}: dataframe shape = {df.shape}")  # iterations x nodes


In [ ]:

# Plot evolution of each metric: one figure with all nodes (thin lines) and highlight top nodes
import matplotlib.cm as cm

def plot_metric_evolution(df, metric_name, out_dir=OUT_DIR):
    plt.figure(figsize=(10,6))
    nodes = df.columns.tolist()
    # plot all nodes with low alpha
    for n in nodes:
        plt.plot(df.index, df[n], linewidth=1, alpha=0.6, label=str(n))
    plt.xlabel('Iteration')
    plt.ylabel(metric_name)
    plt.title(f'Evolution of {metric_name} across iterations (all nodes)')
    plt.grid(True, alpha=0.3)
    # save
    fname = os.path.join(out_dir, f'evolution_{metric_name}_allnodes.png')
    plt.savefig(fname, bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()
    # Also plot top-5 nodes by average value across iterations
    avg = df.mean(axis=0).sort_values(ascending=False)
    top5 = avg.index[:5].tolist()
    plt.figure(figsize=(8,5))
    for n in top5:
        plt.plot(df.index, df[n], marker='o', linewidth=2, label=f'Node {n} (avg={avg[n]:.3f})')
    plt.xlabel('Iteration')
    plt.ylabel(metric_name)
    plt.title(f'Top 5 nodes by average {metric_name} across iterations')
    plt.legend()
    plt.grid(True, alpha=0.3)
    fname2 = os.path.join(out_dir, f'evolution_{metric_name}_top5.png')
    plt.savefig(fname2, bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()

for name,df in metrics_dfs.items():
    plot_metric_evolution(df, name)



## Short discussion / observations

- The recursive spectral modularity bisection finds community structure by repeatedly testing whether a split increases modularity via the leading eigenvalue of the restricted modularity matrix.
- Nodes with high **degree centrality** (many immediate neighbors) tend to remain high-ranked across iterations: these are local hubs that keep many edges inside their community.
- **Betweenness centrality** often highlights bridging nodes that connect communities; during splits, these nodes may increase in relative importance if they sit between emerging communities.
- **Closeness centrality** is influenced by the global distances in the network. As communities split, nodes inside dense clusters may show higher closeness relative to others in their community, but absolute values computed on full graph reflect global reach.
- **Clustering coefficient** is a local measure (triadic closure) and tends to be higher for nodes embedded in tightly-knit groups; it doesn't necessarily change across the recursive splitting because it depends only on local triangles.
- In the Karate Club, typical central nodes (historically known) such as nodes 0 and 33 (the instructor and president) usually remain central by degree and betweenness; the visualizations and metric evolution plots in this notebook make those patterns explicit.

**Caveat:** interpretations above depend on computing metrics on the full graph while labeling changes across iterations. Alternative choices (computing metrics on the induced subgraph per community) will yield different but complementary insights.
